### QED-C Application-Oriented Benchmarks - Hamiltonian Simulation with HamLib - Observables (Customizable)

The notebook contains specific examples for the HamLib-based Hamiltonian Simulation benchmark program.
Configure and run the cell below with the desired execution settings.
Then configure and run the remaining cell(s), each one a variation of this benchmark.

Note: This set of benchmarks surfaces the series of second-level functions used to benchmark HamLib observables.
This is a WORK-IN-PROGRESS and is provided to enable experimentation with alternative techniques for computing observables.


In [1]:
%reload_ext autoreload
%autoreload 2

In [2]:
import numpy as np
from math import sin, cos, pi
import time

# Configure module paths
import sys
sys.path.insert(1, "_common")
sys.path.insert(1, "qiskit")

# Import HamLib helper functions (from _common)
import hamlib_utils

# Import Hamlib Simulation kernel (from qiskit)
import hamlib_simulation_kernel

# Import Observable helper functions
import observables
import evolution_exact

verbose = True

#### for executing circuits to compute observables ...

# Import Qiskit and Qiskit Pauli operator classes
#from qiskit.quantum_info import Pauli, SparsePauliOp
#from qiskit import QuantumCircuit

# Initialize simulator backend
#from qiskit_aer import Aer
#backend = Aer.get_backend('qasm_simulator')


### Configure Options for Execution

In [3]:
# Qubit width of the Hamiltonian
num_qubits = 6

# Number of shots used for execution
num_shots = 10000

# Parameters of Trotterized simulation
K = 1
t = 0.05

# for debugging Hamlib helper functions
hamlib_simulation_kernel.verbose = False
hamlib_utils.verbose = False

# for debugging observables module
observables.verbose_circuits = False


### Specify a Hamiltonian for Testing
Choose a Hamiltonian from HamLib or create a custom Hamiltonian


In [4]:
# Specify the desired Hamiltonian from HamLib 
# Set the hamiltonian_name: 'TFIM', 'Fermi-Hubbard-1D', 'Bose-Hubbard-1D', 'Heisenberg' or 'Max3Sat'

hamiltonian_name = 'TFIM'

print(f"... Hamiltonian name = {hamiltonian_name}")

########### HamLib Hamiltonians

if hamiltonian_name == 'TFIM':
    hamiltonian_params = { "1D-grid": "pbc", "h": 2 }

if hamiltonian_name == 'Heisenberg':
    hamiltonian_params = { "1D-grid": "pbc", "h": 2 }
    
if hamiltonian_name == 'Fermi-Hubbard-1D':
    hamiltonian_params = { "1D-grid": "pbc", "enc": "bk", "U":12 }

if hamiltonian_name == 'Bose-Hubbard-1D':
    hamiltonian_params = { "1D-grid": "nonpbc", "enc": "gray", "U":10 }

if hamiltonian_name == 'Max3Sat':
    hamiltonian_params = { "ratio": "2", "rinst": "02" }

if hamiltonian_name == 'chemistry/electronic/standard/H2':
    hamiltonian_params = { "ham_BK": '' }

# load the HamLib file for the given hamiltonian name
hamlib_utils.load_hamlib_file(filename=hamiltonian_name)

# return a sparse Pauli list of terms queried from the open HamLib file
sparse_pauli_terms, dataset_name = hamlib_utils.get_hamlib_sparsepaulilist(num_qubits=num_qubits, params=hamiltonian_params)
print(f"... dataset_name = {dataset_name}")
print(f"... sparse_pauli_terms = \n{sparse_pauli_terms}")

print("")


... Hamiltonian name = TFIM
... dataset_name = graph-1D-grid-pbc-qubitnodes_Lx-6_h-2
... sparse_pauli_terms = 
[({0: 'Z', 1: 'Z'}, (1+0j)), ({0: 'Z', 5: 'Z'}, (1+0j)), ({1: 'Z', 2: 'Z'}, (1+0j)), ({5: 'Z', 4: 'Z'}, (1+0j)), ({2: 'Z', 3: 'Z'}, (1+0j)), ({3: 'Z', 4: 'Z'}, (1+0j)), ({0: 'X'}, (2+0j)), ({1: 'X'}, (2+0j)), ({5: 'X'}, (2+0j)), ({2: 'X'}, (2+0j)), ({3: 'X'}, (2+0j)), ({4: 'X'}, (2+0j))]



### Group the Pauli Terms and Generate Merged Terms
Here, we optimize the grouping of the Pauli terms, both for creating the base circuit with evolution terms and for generating the measurement circuits.

In [5]:
# DEVNOTE: expand later to include new types of optimizations

# Flag to control optimize by use of commuting groups
use_commuting_groups = True

# group Pauli terms for quantum execution, optionally combining commuting terms into groups.
pauli_term_groups, pauli_str_list = observables.group_pauli_terms_for_execution(
        num_qubits, sparse_pauli_terms, use_commuting_groups)

print(f"... Pauli Term Groups:")
for group in pauli_term_groups:
    print(group)

print(f"\n... Merged Pauli strings, one per group:\n  {pauli_str_list}\n")


... Pauli Term Groups:
[('ZZIIII', (1+0j)), ('ZIIIIZ', (1+0j)), ('IZZIII', (1+0j)), ('IIIIZZ', (1+0j)), ('IIZZII', (1+0j)), ('IIIZZI', (1+0j))]
[('XIIIII', (2+0j)), ('IXIIII', (2+0j)), ('IIIIIX', (2+0j)), ('IIXIII', (2+0j)), ('IIIXII', (2+0j)), ('IIIIXI', (2+0j))]

... Merged Pauli strings, one per group:
  ['ZZZZZZ', 'XXXXXX']



### Create a Hamiltonian Simulation Circuit
Initialize the circuit to a known initial state, append 'K' Trotter steps for total time 't', followed by basis rotation gates.

Optionally, create the base circuit using grouped terms (TBD).

In [6]:
# create Trotterized evolution circuit for HamLib Hamiltonians

init_state = "checkerboard"

qc, _ = hamlib_simulation_kernel.HamiltonianSimulation(
    num_qubits=num_qubits, 
    ham_op=sparse_pauli_terms,
    K=K, t=t,
    init_state = init_state,
    append_measurements = False,
    method = 1, 
)

print(f"... Trotterized Circuit, K={K}, t={t}")
if num_qubits < 11:
    if hamiltonian_name != '':
        hamlib_simulation_kernel.kernel_draw(qc, method=1)
    else:
        print(qc)
else:
    print("  ... circuit is too large to draw.")

print("")


... Trotterized Circuit, K=1, t=0.05
Sample Circuit:
  H = [({0: 'Z', 1: 'Z'}, (1+0j)), ({0: 'Z', 5: 'Z'}, (1+0j)), ({1: 'Z', 2: 'Z'}, (1+0j)), ({5: 'Z', 4: 'Z'}, (1+0j)), ({2: 'Z', 3: 'Z'}, (1+0j)), ({3: 'Z', 4: 'Z'}, (1+0j)), ({0: 'X'}, (2+0j)), ({1: 'X'}, (2+0j)), ({5: 'X'}, (2+0j)), ({2: 'X'}, (2+0j)), ({3: 'X'}, (2+0j)), ({4: 'X'}, (2+0j))]
     ┌────────┐ ░ ┌───────────────┐ ░ 
q_0: ┤0       ├─░─┤0              ├─░─
     │        │ ░ │               │ ░ 
q_1: ┤1       ├─░─┤1              ├─░─
     │        │ ░ │               │ ░ 
q_2: ┤2       ├─░─┤2              ├─░─
     │  Neele │ ░ │  e^-iHt(0.05) │ ░ 
q_3: ┤3       ├─░─┤3              ├─░─
     │        │ ░ │               │ ░ 
q_4: ┤4       ├─░─┤4              ├─░─
     │        │ ░ │               │ ░ 
q_5: ┤5       ├─░─┤5              ├─░─
     └────────┘ ░ └───────────────┘ ░ 
  Initial State Neele:
     ┌───┐
q_0: ┤ X ├
     └───┘
q_1: ─────
     ┌───┐
q_2: ┤ X ├
     └───┘
q_3: ─────
     ┌───┐
q_4: ┤ X ├
     └───┘
q

### Create Measurement Circuits from Base Circuit and Pauli Terms
Here, we append basis rotation gates for each Pauli Term group to the base evolution circuit to create an array of circuits for execution.

In [7]:
# generate an array of circuits, one for each pauli_string in list
circuits = hamlib_simulation_kernel.create_circuits_for_pauli_terms(qc, num_qubits, pauli_str_list)

print(f"... Created {len(circuits)} circuits, one for each group:")               
for circuit, group in list(zip(circuits, pauli_term_groups)):
    print(group)
    #print(circuit)
    

... Created 2 circuits, one for each group:
[('ZZIIII', (1+0j)), ('ZIIIIZ', (1+0j)), ('IZZIII', (1+0j)), ('IIIIZZ', (1+0j)), ('IIZZII', (1+0j)), ('IIIZZI', (1+0j))]
[('XIIIII', (2+0j)), ('IXIIII', (2+0j)), ('IIIIIX', (2+0j)), ('IIXIII', (2+0j)), ('IIIXII', (2+0j)), ('IIIIXI', (2+0j))]


### Execute Circuits to Produce Measurement Distributions
For now, we execute using the Aer simulator. This will be enhanced to use the QED-C execution module to enable execution on multiple programming APIs and backend systems. 

In [8]:
# Initialize simulator backend
from qiskit_aer import Aer
backend = Aer.get_backend('qasm_simulator')

print(f"... begin executing {len(circuits)} circuits ...")
ts = time.time()

# Execute all of the circuits to obtain array of result objects
results = backend.run(circuits, num_shots=num_shots).result()

exec_time = round(time.time()-ts, 3)
print(f"... finished executing {len(circuits)} circuits, total execution time = {exec_time} sec.\n")


... begin executing 2 circuits ...
... finished executing 2 circuits, total execution time = 0.017 sec.



### Compute the Energy of the Evolved State from the Hamiltonian and the Measurement Distributions
Here, we process the measurement results from circuit execution using the Hamiltonian terms to compute the expectation value of the energy observable.

We also, retain the contributions from each terms so that we can use those in calcuating other observables from the same measurement results.

In [9]:
# Compute the total energy for the Hamiltonian

print(f"... begin computing observable value ...")
ts = time.time()

total_energy, term_contributions = observables.calculate_expectation_from_measurements(
                                            num_qubits, results, pauli_term_groups)
obs_time = round(time.time()-ts, 3)
print(f"... finished computing observable value, computation time = {obs_time} sec.\n")

print(f"    Total Energy: {round(np.real(total_energy), 4)}\n")
print(f"    Term Contributions: {term_contributions}\n")


... begin computing observable value ...
... finished computing observable value, computation time = 0.001 sec.

    Total Energy: -5.7227

    Term Contributions: {'ZZIIII': -0.96875, 'ZIIIIZ': -0.962890625, 'IZZIII': -0.966796875, 'IIIIZZ': -0.955078125, 'IIZZII': -0.970703125, 'IIIZZI': -0.96484375, 'XIIIII': -0.00390625, 'IXIIII': -0.021484375, 'IIIIIX': 0.01171875, 'IIXIII': 0.033203125, 'IIIXII': 0.017578125, 'IIIIXI': -0.00390625}



### Compute Energy of the Evolved State Classically and Compare to Quantum Result
Here, we use a classical computation to determine the exact expectation value and produce a quality score for the quantum computation.

In [10]:
print(f"... begin classical computation of expectation value ...")
                    
ts = time.time()

# DEVNOTE: ideally, we can remove this next line somehow      
# create quantum circuit with initial state
qc_initial = hamlib_simulation_kernel.initial_state(n_spins=num_qubits, init_state=init_state)
"""                        
# DEVNOTE: the following code is WIP ... 
# it is for testing the new versions of exact expectation and distribution calculations.
# There is another version in development that does not require the kernel to create an initial circuit.
"""        
correct_exp, correct_dist = evolution_exact.compute_expectation_exact_spo_scipy(
        init_state, 
        qc_initial,
        num_qubits,
        hamlib_simulation_kernel.ensure_sparse_pauli_op(sparse_pauli_terms, num_qubits),
        1.0            # time
        )
    
exact_time = round(time.time()-ts, 3)
print(f"... exact computation time = {exact_time} sec")

print(f"\nExact expectation value, computed classically: {round(np.real(correct_exp), 4)}")
print(f"Estimated expectation value, computed using quantum algorithm: {round(np.real(total_energy), 4)}\n")

simulation_quality = round(np.real(total_energy) / np.real(correct_exp), 3)
print(f"    ==> Simulation Quality: {np.real(simulation_quality)}\n")


... begin classical computation of expectation value ...
... exact computation time = 0.009 sec

Exact expectation value, computed classically: -6.0
Estimated expectation value, computed using quantum algorithm: -5.7227

    ==> Simulation Quality: 0.954



In [11]:
# Flag to control optimize by use of commuting groups
use_commuting_groups = True

N = 3
print(f"\n************ Compute energy of the Hamiltonian {N} times")

# Calculate the total energy
for i in range(N):
        
    # Estimate expectation for the Ham terms using the circuit qc
    total_energy, term_contributions = observables.estimate_expectation_value(backend, qc, sparse_pauli_terms, 
                                             use_commuting_groups=use_commuting_groups, num_shots=10000)
    print(f"\nTotal Energy: {total_energy}")
    print(f"Term Contributions: {term_contributions}")
    
print("")


************ Compute energy of the Hamiltonian 3 times

Total Energy: (-5.8712+0j)
Term Contributions: {'ZZIIII': -0.9602, 'ZIIIIZ': -0.956, 'IZZIII': -0.9622, 'IIIIZZ': -0.9582, 'IIZZII': -0.9568, 'IIIZZI': -0.957, 'XIIIII': -0.023, 'IXIIII': 0.0098, 'IIIIIX': -0.0216, 'IIXIII': -0.0174, 'IIIXII': 0.0118, 'IIIIXI': -0.02}

Total Energy: (-5.7616000000000005+0j)
Term Contributions: {'ZZIIII': -0.9578, 'ZIIIIZ': -0.958, 'IZZIII': -0.9594, 'IIIIZZ': -0.962, 'IIZZII': -0.9584, 'IIIZZI': -0.9624, 'XIIIII': 0.0052, 'IXIIII': -0.0132, 'IIIIIX': -0.0034, 'IIXIII': 0.0168, 'IIIXII': -0.0046, 'IIIIXI': -0.0026}

Total Energy: (-5.814400000000001+0j)
Term Contributions: {'ZZIIII': -0.9618, 'ZIIIIZ': -0.9648, 'IZZIII': -0.9562, 'IIIIZZ': -0.9634, 'IIZZII': -0.9576, 'IIIZZI': -0.9602, 'XIIIII': -0.0162, 'IXIIII': -0.0184, 'IIIIIX': 0.017, 'IIXIII': -0.0124, 'IIIXII': 0.013, 'IIIIXI': -0.0082}



### Compute Expectation with Estimator
Here, we do a sanity check on the above processes by comparing with energy obtained using the Qiskit Estimator.

In [12]:
# Call the Qiskit Estimator using the function wrapper in 'observables' module.

# DEVNOTE: We may want to surface the actual Estimator call instead. 

# Ensure that the pauli_terms are in 'full' format, not 'sparse' - convert if necessary
pauli_terms = observables.ensure_pauli_terms(sparse_pauli_terms, num_qubits=num_qubits)
pauli_terms = observables.swap_pauli_list(pauli_terms)

ts = time.time()

estimator_energy = observables.estimate_expectation_with_estimator(backend, qc, pauli_terms, num_shots=num_shots)

estimator_time = round(time.time()-ts, 3)
print(f"... Estimator computation time = {estimator_time} sec")

print(f"Expectation value, computed using Qiskit Estimator: {round(np.real(estimator_energy), 4)}\n")


... Estimator computation time = 0.005 sec
Expectation value, computed using Qiskit Estimator: -5.7632



### Compute expectation value of other observables.
WORK-IN-PROGRESS: Note that this only works for the Ising model so far.

In [13]:
# Define additional Hamiltonian terms for other Ising observables 
H_terms_spin_correlation = observables.swap_pauli_list([(0.2,'IIIIZZ'), (0.2,'IIIZZI'), (0.2,'IIZZII'), (0.2,'IZZIII'), (0.2,'ZZIIII')])
H_terms_magnetization = observables.swap_pauli_list([(1,'IIIIIZ'), (1,'IIIIZI'), (1,'IIIZII'), (1,'IIZIII'), (1,'IZIIII'), (1, 'ZIIIII')])

spin_correlation = observables.calculate_expectation_from_contributions(term_contributions, H_terms_spin_correlation)
print(spin_correlation)

magnetization = observables.calculate_expectation_from_contributions(term_contributions, H_terms_magnetization)
print(magnetization)


-0.9598400000000001
WARN: term not found in term_contributions: IIIIIZ
WARN: term not found in term_contributions: IIIIZI
WARN: term not found in term_contributions: IIIZII
WARN: term not found in term_contributions: IIZIII
WARN: term not found in term_contributions: IZIIII
WARN: term not found in term_contributions: ZIIIII
0
